In [1]:
#Importing all the neccessary libraries
import pandas as pd
import numpy as np

#Libraries for visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import sweetviz as sv

#Libraries for formattting and handling text 
import string
import re

#Library for nltk
import nltk 
import nltk.corpus
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer

from collections import defaultdict
from collections import  Counter
stop=set(stopwords.words('english'))

#Library for Splitting Dataset
from sklearn.model_selection import train_test_split

#Library for TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer

#Sklearn ml models
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
import time
#Library for evaluation
from sklearn.metrics import accuracy_score, f1_score

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VovAz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\VovAz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VovAz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df=pd.read_csv('C:/Users/VovAz/OneDrive/Desktop/Проект/Court/justice.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3303 entries, 0 to 3302
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          3303 non-null   int64 
 1   ID                  3303 non-null   int64 
 2   name                3303 non-null   object
 3   href                3303 non-null   object
 4   docket              3303 non-null   object
 5   term                3303 non-null   object
 6   first_party         3302 non-null   object
 7   second_party        3302 non-null   object
 8   facts               3303 non-null   object
 9   facts_len           3303 non-null   int64 
 10  majority_vote       3303 non-null   int64 
 11  minority_vote       3303 non-null   int64 
 12  first_party_winner  3288 non-null   object
 13  decision_type       3296 non-null   object
 14  disposition         3231 non-null   object
 15  issue_area          3161 non-null   object
dtypes: int64(5), object(11)


In [4]:
print('Sample text from the train dataset:')
df['facts'].head()

Sample text from the train dataset:


0    <p>In 1970, Jane Roe (a fictional name used in...
1    <p>Joan Stanley had three children with Peter ...
2    <p>John Giglio was convicted of passing forged...
3    <p>The Idaho Probate Code specified that "male...
4    <p>Miller, after conducting a mass mailing cam...
Name: facts, dtype: object

In [5]:
report_con = sv.analyze([df, 'first_party'])

                                             |          | [  0%]   00:00 -> (? left)

In [6]:
report_con.show_html()

Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Data is pretty full, there is not many missing values, key features and target missing around 0.5% of data. In secondary features there is up to 5% of missing data.

In [7]:
def clean_text(text):
    text=str(text).lower() #converts text to lowercase
    text=re.sub('\d+', '', text) #removes numbers
    text = text.translate(str.maketrans('', '', string.punctuation))    #removes punctuations
    return text

In [8]:
df['clean_text']=df['facts'].apply(clean_text)# apply a clean_text function on 'text' row 
df.head(10)# checking


,Unnamed: 0,ID,name,href,docket,term,first_party,second_party,facts,facts_len,majority_vote,minority_vote,first_party_winner,decision_type,disposition,issue_area,clean_text
0,0,50606,Roe v. Wade,https://api.oyez.org/cases/1971/70-18,70-18,1971,Jane Roe,Henry Wade,"<p>In 1970, Jane Roe (a fictional name used in...",501,7,2,True,majority opinion,reversed,NaN,pin jane roe a fictional name used in court d...
1,1,50613,Stanley v. Illinois,https://api.oyez.org/cases/1971/70-5014,70-5014,1971,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,757,5,2,True,majority opinion,reversed/remanded,Civil Rights,pjoan stanley had three children with peter st...
2,2,50623,Giglio v. United States,https://api.oyez.org/cases/1971/70-29,70-29,1971,John Giglio,United States,<p>John Giglio was convicted of passing forged...,495,7,0,True,majority opinion,reversed/remanded,Due Process,pjohn giglio was convicted of passing forged m...
3,3,50632,Reed v. Reed,https://api.oyez.org/cases/1971/70-4,70-4,1971,Sally Reed,Cecil Reed,"<p>The Idaho Probate Code specified that ""male...",378,7,0,True,majority opinion,reversed/remanded,Civil Rights,pthe idaho probate code specified that males m...
4,4,50643,Miller v. California,https://api.oyez.org/cases/1971/70-73,70-73,1971,Marvin Miller,California,"<p>Miller, after conducting a mass mailing cam...",305,5,4,True,majority opinion,vacated/remanded,First Amendment,pmiller after conducting a mass mailing campai...
5,5,50644,Kleindienst v. Mandel,https://api.oyez.org/cases/1971/71-16,71-16,1971,"Richard G. Kleindienst, Attorney General of th...","Ernest E. Mandel, et al.",<p>Ernest E. Mandel was a Belgian professional...,2282,6,3,True,majority opinion,reversed,First Amendment,pernest e mandel was a belgian professional jo...
6,6,50655,Sarno v. Illinois Crime Investigating Commission,https://api.oyez.org/cases/1971/70-7,70-7,1971,Sarno,Illinois Crime Investigating Commission,<p>The Illinois Crime Investigating Commission...,1424,5,2,False,per curiam,NaN,Criminal Procedure,pthe illinois crime investigating commission w...
7,7,50656,Argersinger v. Hamlin,https://api.oyez.org/cases/1971/70-5015,70-5015,1971,Argersinger,Hamlin,<p>Jon Argersinger was an indigent charged wit...,347,9,0,True,majority opinion,reversed,Criminal Procedure,pjon argersinger was an indigent charged with ...
8,8,50657,Eisenstadt v. Baird,https://api.oyez.org/cases/1971/70-17,70-17,1971,Eisenstadt,Baird,<p>William Baird gave away Emko Vaginal Foam t...,420,6,1,False,majority opinion,affirmed,Privacy,pwilliam baird gave away emko vaginal foam to ...
9,9,50663,Gooding v. Wilson,https://api.oyez.org/cases/1971/70-26,70-26,1971,Gooding,Wilson,<p>A Georgia state court convicted Johnny Wils...,612,5,2,False,majority opinion,affirmed,First Amendment,pa georgia state court convicted johnny wilson...


In [9]:
# removing stopwords using nltk
df['clean_text'] = df['clean_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df.head()



,Unnamed: 0,ID,name,href,docket,term,first_party,second_party,facts,facts_len,majority_vote,minority_vote,first_party_winner,decision_type,disposition,issue_area,clean_text
0,0,50606,Roe v. Wade,https://api.oyez.org/cases/1971/70-18,70-18,1971,Jane Roe,Henry Wade,"<p>In 1970, Jane Roe (a fictional name used in...",501,7,2,True,majority opinion,reversed,NaN,pin jane roe fictional name used court documen...
1,1,50613,Stanley v. Illinois,https://api.oyez.org/cases/1971/70-5014,70-5014,1971,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,757,5,2,True,majority opinion,reversed/remanded,Civil Rights,pjoan stanley three children peter stanley sta...
2,2,50623,Giglio v. United States,https://api.oyez.org/cases/1971/70-29,70-29,1971,John Giglio,United States,<p>John Giglio was convicted of passing forged...,495,7,0,True,majority opinion,reversed/remanded,Due Process,pjohn giglio convicted passing forged money or...
3,3,50632,Reed v. Reed,https://api.oyez.org/cases/1971/70-4,70-4,1971,Sally Reed,Cecil Reed,"<p>The Idaho Probate Code specified that ""male...",378,7,0,True,majority opinion,reversed/remanded,Civil Rights,pthe idaho probate code specified males must p...
4,4,50643,Miller v. California,https://api.oyez.org/cases/1971/70-73,70-73,1971,Marvin Miller,California,"<p>Miller, after conducting a mass mailing cam...",305,5,4,True,majority opinion,vacated/remanded,First Amendment,pmiller conducting mass mailing campaign adver...


In [10]:
lemma=WordNetLemmatizer()
#transform clear text to lemma
def lemmatizer(text):
    text= ' '.join(lemma.lemmatize(word) for word in text.split(' '))
    return text

df['final_text']=df['clean_text'].apply(lemmatizer)
df.head()
df1 = df.loc[:, ["final_text","first_party_winner"]]# leaving features and target columns
df1.dropna(inplace = True)#drop missing values
print('We have lost {:.2%} of data'.format(1 - (len(df1))/len(df)))
len(df1)

We have lost 0.45% of data


3288

### We cleared the text from punctuations, stopwords and transformed it to lemma. Now it's time to train and test our models.

In [11]:
X=df1['final_text']
y=df1['first_party_winner'].values.astype(int)#change type object to integer

In [12]:
from sklearn.model_selection import train_test_split

X_train1,X_test,y_train1,y_test=train_test_split(X,y,test_size= 0.2, random_state= 123456) #splitting to train, valid and test data
X_train,X_valid,y_train,y_valid=train_test_split(X_train1,y_train1,test_size= 0.25, random_state= 123456,stratify=y_train1)


In [13]:
tfidf=TfidfVectorizer()

X_train=tfidf.fit_transform(X_train).toarray()#transforming text data to vector
X_train1 = tfidf.transform(X_train1).toarray()
X_valid=tfidf.transform(X_valid).toarray()


In [14]:
train_start = time.time()
model = RandomForestClassifier() 
hyperparams = [{'n_estimators' : [x for x in range(1, 10)], 
                'max_depth':[x for x in range(1, 20)], 
                'random_state':[123456]}]

clf = GridSearchCV(model, hyperparams, scoring='f1')
clf.fit(X_train, y_train)
print("Best hyperparams:")

print(clf.best_params_)
train_end = time.time()
print('Tuning hyperparams time:{}'.format(train_end - train_start))

Best hyperparams:
{'max_depth': 6, 'n_estimators': 8, 'random_state': 123456}
Tuning hyperparams time:210.4280595779419


In [15]:
train_start = time.time()
model = LGBMClassifier() 

hyperparams = [{'num_leaves' : [x for x in range(1, 102, 20)], 
                'max_depth':[x for x in range(1, 12, 2)],
                'random_state':[123456]}]


clf = GridSearchCV(model, hyperparams, scoring='f1')
clf.fit(X_train, y_train)
print("Best hyperparams:")

print(clf.best_params_)
train_end = time.time()
print('Tuning hyperparams time:{}'.format(train_end - train_start))

Best hyperparams:
{'max_depth': 1, 'num_leaves': 21, 'random_state': 123456}
Tuning hyperparams time:89.25588011741638


In [16]:
train_start = time.time()
model = DecisionTreeClassifier() 

hyperparams = [{'max_depth':[x for x in range(1, 20)], 
                'random_state':[123456]}]


clf = GridSearchCV(model, hyperparams, scoring='f1')
clf.fit(X_train, y_train)
print("Best hyperparams:")

print(clf.best_params_)
train_end = time.time()
print('Tuning hyperparams time:{}'.format(train_end - train_start))

Best hyperparams:
{'max_depth': 1, 'random_state': 123456}
Tuning hyperparams time:269.73597240448


In [17]:
models = [RandomForestClassifier(max_depth = 6, n_estimators = 8, random_state = 123456), 
          DecisionTreeClassifier(max_depth = 1, random_state= 123456),
          LGBMClassifier(max_depth= 1, num_leaves = 21, random_state= 123456)]


results = []

for model in models: 
    train_start = time.time()  
    model.fit(X_train, y_train)
    train_end = time.time()
    predict_start = time.time()
    pred = model.predict(X_valid)
    predict_end = time.time()
    f1 = f1_score(y_valid, pred)
    results.append({'model_name': model.__class__.__name__, 
                    'training time': train_end - train_start,
                    'prediction time': predict_end - predict_start,
                    'f1': f1                             
                              })  
              
res = pd.DataFrame(results)
res

,model_name,training time,prediction time,f1
0,RandomForestClassifier,0.193203,0.054565,0.787097
1,DecisionTreeClassifier,0.565926,0.031246,0.789328
2,LGBMClassifier,0.416988,0.015626,0.790353


Tuning hyperparams shows best models, let's test it.

In [18]:
test_frame = tfidf.transform(X_test).toarray()
model = RandomForestClassifier(max_depth= 6, n_estimators = 8, random_state= 123456)
model.fit(X_train1,y_train1)
pred = model.predict(test_frame)
print('Random forest F1-score:', f1_score(y_test, pred))
print('Random forest accuracy score: ', accuracy_score(y_test,pred))

Random forest F1-score: 0.7870370370370371
Random forest accuracy score:  0.6504559270516718


In [19]:
model = LGBMClassifier(max_depth= 1, num_leaves = 21, random_state= 123456)
model.fit(X_train1,y_train1)
pred = model.predict(test_frame)
print('LGBM F1-score:', f1_score(y_test, pred))
print('LGBM accuracy score:', accuracy_score(y_test,pred))

LGBM F1-score: 0.7881591119333949
LGBM accuracy score: 0.6519756838905775


In [20]:
model = DecisionTreeClassifier(max_depth= 1, random_state= 123456)
model.fit(X_train1,y_train1)
pred = model.predict(test_frame)
print('DecisionTreeClassifier f1-score', f1_score(y_test, pred))
print('DecisionTreeClassifier accuracy score', accuracy_score(y_test,pred))

DecisionTreeClassifier f1-score 0.7870967741935483
DecisionTreeClassifier accuracy score 0.648936170212766


LGBM shows best results, but not so far from Random forest and decicsion tree. All three models can be used at production.